# Generate sample data

In [6]:
import random

first_names = ["Ali", "Zahra", "Reza", "Sara", "Mohammad", "Fatemeh", "Hossein", "Maryam", "Mehdi", "Narges", "Hamed", "Roya"]
last_names = ["Ahmadi", "Hosseini", "Karimi", "Rahimi", "Hashemi", "Ebrahimi", "Moradi", "Mohammadi", "Rostami", "Fazeli", "Hosseinzadeh", "Niknam"]

random.seed(42)
# Generate unique random names
random_names = set()
while len(random_names) < 24:
    first_name = random.choice(first_names)
    last_name = random.choice(last_names)
    random_names.add(f"{first_name} {last_name}")

random_names = list(random_names)  # Convert set to list

random_names

['Zahra Rahimi',
 'Mohammad Rahimi',
 'Mohammad Ahmadi',
 'Hamed Niknam',
 'Sara Karimi',
 'Zahra Hosseini',
 'Sara Ebrahimi',
 'Mehdi Hosseini',
 'Hamed Hosseini',
 'Roya Hosseinzadeh',
 'Ali Niknam',
 'Narges Moradi',
 'Mohammad Karimi',
 'Reza Niknam',
 'Mehdi Rahimi',
 'Roya Hosseini',
 'Maryam Fazeli',
 'Sara Rostami',
 'Ali Ahmadi',
 'Roya Rostami',
 'Hossein Ebrahimi',
 'Narges Ahmadi',
 'Hossein Hosseini',
 'Hossein Rahimi']

# Generate conflict matrix

In [7]:
import numpy as np
import pandas as pd

# Initialize a 24x24 matrix with zeros
conflict_matrix = np.zeros((24, 24), dtype=int)

# Randomly set 40 unique pairs for conflicts
number_conf = 0
while number_conf < 40:
    i = random.randint(0, 23)
    j = random.randint(0, 23)
    if i != j and conflict_matrix[i, j]!=1:
        conflict_matrix[i, j] = 1
        number_conf+=1

# Create a DataFrame with the conflict matrix
conflict_df = pd.DataFrame(conflict_matrix, index=random_names, columns=random_names)
#print(conflict_df)
print("number of conflict: ", np.count_nonzero(conflict_df == 1))


number of conflict:  40


In [8]:
conflict_matrix[16]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class SeatingNetwork(nn.Module):
    def __init__(self):
        super(SeatingNetwork, self).__init__()
        self.fc1 = nn.Linear(24 * 24, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 24)  # Output 24 values
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        x = x.view(4, 6)  # Reshape the output to 4x6
        return x

In [27]:

def calculate_total_conflict(seating_arrangement, conflict_matrix):
    total_conflict = 0.0
    for i in range(4):
      for j in range(6):
        person = seating_arrangement[i][j]
        person_n_i = i*6 + j
        if i>0:
          person_up = seating_arrangement[i-1][j]
          person_n_j = (i-1)*6 + j
          total_conflict += conflict_matrix[person_n_i][person_n_j] * torch.abs(person - person_up)

        if i<3:
          person_bottom = seating_arrangement[i+1][j]
          person_n_j = (i+1)*6 + j
          total_conflict += conflict_matrix[person_n_i][person_n_j] * torch.abs(person - person_bottom)

        if j>0:
          person_left = seating_arrangement[i][j-1]
          person_n_j = i*6 + (j-1)
          total_conflict += conflict_matrix[person_n_i][person_n_j] * torch.abs(person - person_left)

        if j<5:
          person_right = seating_arrangement[i][j+1]
          person_n_j = i*6 + (j+1)
          total_conflict += conflict_matrix[person_n_i][person_n_j] * torch.abs(person - person_right)

    return total_conflict


def custom_loss_function(y_pred, conflict_matrix):
    seating_arrangement = y_pred
    total_conflict = calculate_total_conflict(seating_arrangement, conflict_matrix)
    return total_conflict


# Train

In [28]:

conflict_matrix_tensor = torch.tensor(conflict_matrix, dtype=torch.float32)

# Define the model and optimizer
model = SeatingNetwork()
optimizer = optim.Adam(model.parameters(), lr=0.001)

per_loss = 1
for epoch in range(2000):
    y_pred = model(conflict_matrix_tensor)
    y_pred.requires_grad_(True)

    loss = custom_loss_function(y_pred, conflict_matrix_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #if epoch % 10 == 0:
    print(f'Epoch {epoch}, Loss: {loss.item()}')
    if per_loss<loss.item() or loss==0:
      break
    per_loss = loss
print("Training complete")


Epoch 0, Loss: 0.24939480423927307
Epoch 1, Loss: 0.31881844997406006
Training complete


# predict

In [33]:
import numpy as np

def count_conflicts(seating_arrangement, conflict_matrix):
    total_conflict = 0

    for i in range(4):
      for j in range(6):
        person = seating_arrangement[i][j]
        person_n_i = i*6 + j
        if i>0:
          person_up = seating_arrangement[i-1][j]
          person_n_j = (i-1)*6 + j
          if conflict_matrix[person][person_up]==1:
            print(person, person_up)
            total_conflict += 1

        if i<3:
          person_bottom = seating_arrangement[i+1][j]
          person_n_j = (i+1)*6 + j
          if conflict_matrix[person][person_bottom]==1:
            print(person, person_bottom)
            total_conflict += 1

        if j>0:
          person_left = seating_arrangement[i][j-1]
          person_n_j = i*6 + (j-1)
          if conflict_matrix[person][person_left]==1:
            print(person, person_left)
            total_conflict += 1

        if j<5:
          person_right = seating_arrangement[i][j+1]
          person_n_j = i*6 + (j+1)
          if conflict_matrix[person][person_right]==1:
            print(person, person_right)
            total_conflict += 1
    return total_conflict

In [34]:
# Make predictions
with torch.no_grad():
    model.eval()
    y_pred = model(conflict_matrix_tensor)
    predicted_seating_arrangement = y_pred.numpy()
    print(predicted_seating_arrangement)

[[-0.05765697  0.01802682  0.04081899 -0.02138504 -0.04815894  0.00678154]
 [-0.01760353  0.02048022  0.0407415   0.00597371  0.08375765 -0.00309765]
 [-0.04335405 -0.02195673 -0.06520856  0.04646108 -0.05045597 -0.05848679]
 [-0.00399736 -0.02234163  0.03219848 -0.04365114  0.03085902 -0.03783605]]


In [35]:
sorted_indices = np.argsort(predicted_seating_arrangement, axis=None)

sorted_positions = np.arange(predicted_seating_arrangement.size)[sorted_indices]

predicted_seating_arrangement = sorted_positions.reshape(predicted_seating_arrangement.shape)

print("Predicted seating arrangement:\n", predicted_seating_arrangement)

total_conflicts = count_conflicts(predicted_seating_arrangement, conflict_matrix)
print(f'\n Total number of conflicts in the predicted arrangement: {total_conflicts}')


Predicted seating arrangement:
 [[14 17  0 16  4 21]
 [12 23 19 13  3  6]
 [18 11  9  5  1  7]
 [22 20  8  2 15 10]]
14 17
18 12
18 22
1 7
20 11
20 22

 Total number of conflicts in the predicted arrangement: 6


In [36]:
conflict_matrix[14]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0])